<a href="https://colab.research.google.com/github/kkoehn8/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-06 02:58:34--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.53MB/s    in 0.2s    

2022-10-06 02:58:35 (5.53 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
#Connect to my Outdoors Vine Review csv file
from pyspark import SparkFiles
url = "<my url>"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Vine_data.csv"), sep=",", header=True, inferSchema=True)
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [6]:
#Select rows
df_reviews = df.select(["star_rating", "helpful_votes", "total_votes", "vine","verified_purchase"])

#Filter for total votes greater than 20
df_reviews_20 = df_reviews.filter(df_reviews["total_votes"]>=20)
df_reviews_20.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           25|         29|   N|                Y|
|          5|           31|         31|   N|                Y|
|          3|           29|         31|   N|                Y|
|          3|           26|         27|   N|                Y|
|          3|           20|         21|   N|                Y|
|          5|           35|         39|   N|                Y|
|          4|           26|         26|   N|                Y|
|          2|           14|         43|   N|                Y|
|          1|           89|         96|   N|                Y|
|          5|           22|         22|   N|                Y|
|          5|           27|         27|   N|                Y|
|          4|           21|         21|   N|                Y|
|          5|           26|         30|   N|           

In [8]:
#Filter for helpful votes divided by total votes greater than or equal to 50%
df_reviews_50 = df_reviews_20.filter(df_reviews_20["helpful_votes"]/df_reviews_20["total_votes"] >=0.50)
df_reviews_50.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           25|         29|   N|                Y|
|          5|           31|         31|   N|                Y|
|          3|           29|         31|   N|                Y|
|          3|           26|         27|   N|                Y|
|          3|           20|         21|   N|                Y|
|          5|           35|         39|   N|                Y|
|          4|           26|         26|   N|                Y|
|          1|           89|         96|   N|                Y|
|          5|           22|         22|   N|                Y|
|          5|           27|         27|   N|                Y|
|          4|           21|         21|   N|                Y|
|          5|           26|         30|   N|                Y|
|          5|           52|         55|   N|           

In [9]:
#Filter the above that are in the vine program
paid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "Y")
paid_reviews_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           25|         25|   Y|                N|
|          5|           26|         28|   Y|                N|
|          5|           79|         82|   Y|                N|
|          5|           12|         20|   Y|                N|
|          5|           32|         37|   Y|                N|
|          5|           28|         33|   Y|                N|
|          4|           59|         65|   Y|                N|
|          4|           37|         38|   Y|                N|
|          2|           15|         21|   Y|                N|
|          5|          123|        128|   Y|                N|
|          4|           31|         33|   Y|                N|
|          5|           34|         38|   Y|                N|
|          5|           35|         37|   Y|           

In [10]:
#Filter the above that are not in the vine program
unpaid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "N")
unpaid_reviews_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           25|         29|   N|                Y|
|          5|           31|         31|   N|                Y|
|          3|           29|         31|   N|                Y|
|          3|           26|         27|   N|                Y|
|          3|           20|         21|   N|                Y|
|          5|           35|         39|   N|                Y|
|          4|           26|         26|   N|                Y|
|          1|           89|         96|   N|                Y|
|          5|           22|         22|   N|                Y|
|          5|           27|         27|   N|                Y|
|          4|           21|         21|   N|                Y|
|          5|           26|         30|   N|                Y|
|          5|           52|         55|   N|           

In [12]:
#Paid Reviews with 5 star rating
paid_5star_reviews = paid_reviews_df[paid_reviews_df["star_rating"] == 5].count()
paid_5star_reviews

56

In [13]:
#Paid Reviews Total
paid_reviews_total = paid_reviews_df.count()
paid_reviews_total

107

In [14]:
#Percentage of paid 5 star reviews
percentage_paid_5star = paid_5star_reviews / paid_reviews_total * 100
percentage_paid_5star

52.336448598130836

In [15]:
#Unpaid Reviews with 5 star rating
unpaid_5star_reviews = unpaid_reviews_df[unpaid_reviews_df["star_rating"] == 5].count()
unpaid_5star_reviews

21005

In [16]:
#Unpaid Reviews Total
unpaid_reviews_total = unpaid_reviews_df.count()
unpaid_reviews_total

39869

In [17]:
#Percentage of unpaid 5 star reviews
percentage_unpaid_5star = unpaid_5star_reviews / unpaid_reviews_total * 100
percentage_unpaid_5star

52.68504351751988